In [1]:
!pip install datasets -q
!pip install evaluation -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [4]:
import tensorflow as tf

import pandas as pd

import numpy as np

import evaluation

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

from datasets import load_dataset, Dataset

import warnings

warnings.filterwarnings('ignore')

In [5]:
checkpoint = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [6]:
raw_dataset = load_dataset("SetFit/emotion")
raw_dataset


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 2000
    })
})

In [12]:
def tokenize_function(example):
  return tokenizer(example['text'], truncation = True)

In [13]:
tokenized_dataset = raw_dataset.map(tokenize_function, batched = True)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [16]:

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer = tokenizer, return_tensors = 'tf')

# default return tensor is pytorch

In [17]:
tokenized_dataset['train']

Dataset({
    features: ['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 16000
})

In [19]:
train_data = tokenized_dataset['train'].to_tf_dataset(
    batch_size = 64,
    collate_fn = data_collator,
    columns = ['input_ids', 'token_type_ids', 'attention_mask'],
    label_cols = ['label'],
    shuffle = True
)

eval_data = tokenized_dataset['validation'].to_tf_dataset(
    batch_size = 64,
    collate_fn = data_collator,
    columns = ['input_ids', 'token_type_ids', 'attention_mask'],
    label_cols = ['label'],
    shuffle = False
)


test_data = tokenized_dataset['test'].to_tf_dataset(
    batch_size = 64,
    collate_fn = data_collator,
    columns = ['input_ids', 'token_type_ids', 'attention_mask'],
    label_cols = ['label'],
    shuffle = False
)

In [22]:
from huggingface_hub import notebook_login

notebook_login()


In [33]:
from transformers import TFAutoModel
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers.schedules import PolynomialDecay




In [40]:
model = TFAutoModel.from_pretrained("bert-base-uncased")
inputs = tokenizer(['Hello world', 'Hi how are you'], padding=True, truncation=True,
                  return_tensors='tf')
inputs


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

{'input_ids': <tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[ 101, 7592, 2088,  102,    0,    0],
       [ 101, 7632, 2129, 2024, 2017,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[1, 1, 1, 1, 0, 0],
       [1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [41]:
output = model(inputs)
output


TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(2, 6, 768), dtype=float32, numpy=
array([[[-0.1688833 ,  0.13606358, -0.13940008, ..., -0.6251125 ,
          0.05217254,  0.36714533],
        [-0.36327448,  0.14121935,  0.8799882 , ...,  0.10433007,
          0.28875723,  0.3726793 ],
        [-0.6985944 , -0.69879735,  0.06450275, ..., -0.22103691,
          0.00986822, -0.5939794 ],
        [ 0.8309832 ,  0.1236674 , -0.1511907 , ...,  0.10309584,
         -0.67792666, -0.26285216],
        [-0.40266573, -0.01928258,  0.57325053, ..., -0.2065685 ,
          0.0233856 ,  0.2012629 ],
        [-0.6228409 , -0.27453488,  0.18117625, ..., -0.12944855,
         -0.03839095, -0.057332  ]],

       [[ 0.0928655 , -0.02636387, -0.12239331, ..., -0.21063554,
          0.17386377,  0.17250937],
        [ 0.40742067, -0.05930961,  0.5523465 , ..., -0.6790571 ,
          0.65557456, -0.29456607],
        [-0.21155313, -0.68586326, -0.46280748, ...,  0.15278433

In [37]:
output[1]

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.9061534 , -0.31115308, -0.62165374, ..., -0.3057521 ,
        -0.64009386,  0.9166176 ],
       [-0.9309669 , -0.3380703 , -0.6216152 , ..., -0.44018888,
        -0.68128836,  0.93488973]], dtype=float32)>

In [46]:
class BERTForClassification(tf.keras.Model):

    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)


In [47]:
# Assuming checkpoint and other variables are defined properly
model = TFAutoModel.from_pretrained(checkpoint)
classifier = BERTForClassification(model, num_classes=6)


classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [48]:
history = classifier.fit(
    train_data,
    epochs=3,
    verbose = True
)


Epoch 1/3
250/250 [==============================] - 254s 675ms/step - loss: 0.8909 - accuracy: 0.6784
Epoch 2/3
250/250 [==============================] - 163s 651ms/step - loss: 0.1765 - accuracy: 0.9314
Epoch 3/3
250/250 [==============================] - 162s 649ms/step - loss: 0.0966 - accuracy: 0.9553


In [49]:
classifier.evaluate(test_data)

32/32 [==============================] - 17s 231ms/step - loss: 0.1780 - accuracy: 0.9185


[0.17802263796329498, 0.9185000061988831]